## Graph Representation
A graph is represented by  
$$G = (V, E)$$  
Here, $V$ is a set of all vertices  
$$V = \{v_0, v_1, ..., v_k \}$$  
And, $E$ is set of ordered pairs of vertices called as edge. An edge is represented as  
$$(i, j)\ where\ i,j\ \in V$$  
Therefore,  
$$E = \{(v_a, v_b), ..., (v_x, v_y)\}$$  
A path is a sequence of vertices $v_0, v_1, ..., v_k$ where for every $i \in \{0, 1, ..., k\}$, the edge $(v_{i-1}, v_i) \in E$. This path is cyclic if $(v_k, v_0)$ is also present in $E$.  

Also, let $n$ be the number of vertices (order of graph) and $m$ be the number of edges (size of graph).

### Common Operations
- **addEdge(i, j):** add $(i,j)$ to $E$
- **removeEdge(i, j):** remove $(i,j)$ from $E$
- **hasEdge(i, j):** check if edge $(i,j) \in E$
- **inEdge(i):** return set of all $j$ where $(j,i) \in E$
- **outEdge(i):** return set of all $j$ where $(i,j) \in E$

### Adjacency Matrix
Adjacency Matrix is graph representation through a matrix. We create a $n\times n$ boolean matrix. If `array[i][j] == true` this means that $(i,j) \in E$. In case of a undirected graph, the matrix will be symmetrical along the diagonal.  
![Adj Matrix](images/qXMwUGq.png)

In [ ]:
private int n;
private boolean[][] matrix;

public AdjacencyMatrix(int n) {
    this.n = n;
    matrix = new boolean[n][n];
}

public void addEdge(int i, int j) {
    matrix[i][j] = true;
}

public void removeEdge(int i, int j) {
    matrix[i][j] = false;
}

public boolean hasEdge(int i, int j) {
    return matrix[i][j];
}

All the above operations take $O(1)$ time. Adjacency matrix however performs poorly for operations `inEdge` and `outEdge`. It takes $O(V)$ time for both

In [ ]:
List<Integer> inEdge(int i){
    List<Integer> list = new ArrayList<Integer>();
    for(int j=0; j<n; j++) {
        if(matrix[j][i])
            list.add(j);
    }
    return list;
}

List<Integer> outEdge(int i){
    List<Integer> list = new ArrayList<Integer>();
    for(int j=0; j<n; j++) {
        if(matrix[i][j])
            list.add(j);
    }
    return list;
}

The space used by matrix is $O(n^2)$. To conclude, graph creation is faster using adjacency matrix, graph traversal however would be not so fast.

### Adjacency Matrix Property
Let us represent adjacency matrix as $A$.  
![adj matrix](images/k54OSd6.png)  

$$A= \left(\begin{matrix}0&1&0&1&1\\0&0&0&1&0\\0&0&0&0&1\\0&0&0&0&0\\0&1&0&0&0\end{matrix}\right)$$  
$A^2$ represents the matrix of number of 2 length edges from $i$ to $j$.  
$$A^2= \left(\begin{matrix}0&1&0&1&0\\0&0&0&0&0\\0&1&0&0&0\\0&0&0&0&0\\0&0&0&1&0\end{matrix}\right)$$  
Similarly, $A^3$ represents the matrix of number of 3 length edges from $i$ to $j$.  
$$A^3= \left(\begin{matrix}0&0&0&1&0\\0&0&0&0&0\\0&0&0&1&0\\0&0&0&0&0\\0&0&0&0&0\end{matrix}\right)$$  

![adj matrix 2](images/07OflVc.png)  

$$A= \left(\begin{matrix}0&1&1&1\\1&0&1&1\\1&1&0&1\\1&1&1&0\end{matrix}\right)$$  
$$A^3= \left(\begin{matrix}6&7&7&7\\7&6&7&7\\7&7&6&7\\7&7&7&6\end{matrix}\right)$$  

This property can help us identify if there is any cycle in a directed graph or not. We calculate $A^k$, where $n=2,...,n$. If we find that any diagonal element of the matrix is non-zero, we can conclude that the graph is cyclic.

In [ ]:
public boolean isCyclic() {
    SimpleMatrix m = new SimpleMatrix(convert(matrix));
    SimpleMatrix mPower = m;
    for(int i=2; i<=n; i++) {
        mPower = mPower.mult(m);
        if(mPower.trace() != 0.0)
            return true;
    }
    return false;
}

private double[][] convert(boolean[][] input) {
    double[][] result = new double[n][n];
    for(int i=0; i<n; i++) {
        for(int j=0; j<n; j++) {
            if(input[i][j])
                result[i][j] = 1.0;
            else
                result[i][j] = 0.0;
        }
    }
    return result;
}

### Adjacency List
In a adjacency list we maintain a list for every vertex. This vertex contains the list of all vertices connected to it.  
![Adj List](images/vjeEDHu.png)

In [ ]:
private int n;
List<Integer>[] adj;

public AdjacencyList(int n) {
    adj = (List<Integer>[]) new List[n];
    for(int i=0; i<n; i++) {
        adj[i] = new ArrayList<Integer>();
    }
}

public void addEdge(int i, int j) {
    adj[i].add(j);
}

public void removeEdge(int i, int j) {
    Iterator<Integer> iterator = adj[i].iterator();
    while(iterator.hasNext()) {
        if(iterator.next() == j) {
            iterator.remove();
            return;
        }
    }
}

public boolean hasEdge(int i, int j) {
    return adj[i].contains(j);
}

public List<Integer> inEdges(int i) {
    List<Integer> list = new ArrayList<Integer>();
    for(int j=0; j<n; j++) {
        if(adj[j].contains(i))
            list.add(j);
    }
    return list;
}

public List<Integer> outEdges(int i) {
    return adj[i];
}

- `addEdge` takes $O(1)$ time
- `removeEdge` takes $O(deg(i))$ time, where $deg(i)$ counts the number of edges in $E$ that have $i$ as their source  
- `inEdges` takes $O(n+m)$ time  
- `outEdges` takes $O(1)$ time  

Space complexity is $O(n+m)$

## Graph Traversal
**Breadth First Search** for a graph is generalization of the algorithm for a BST.

In [ ]:
public void bfs(int i, IntConsumer c) {
    // this below array required because a node can
    // be reached in more than way
    boolean[] visited = new boolean[n];
    Queue<Integer> q = new ArrayDeque<Integer>();
    q.offer(i);
    visited[i] = true;
    while(!q.isEmpty()) {
        int value = q.poll();
        c.accept(value);
        for(Integer x: outEdges(value)) {
            if(!visited[x]) {
                q.offer(x);
                visited[x] = true;
            }
        }
    }
}

For the example above the sequence is `0->1->3->4->2->5->6`. Breadth first traversal gives us the shortest path between two nodes in an unweighted directed or undirected graph.

Note the usage of visited array. In case of undirected graph or directed cyclic graph, not using it will lead to infinite loop. Even in case of directed acyclic graph, node may get repeated (even though no infinite loop in this case).

**Depth First Search** we divide graph vertices into three type: GRAY(currently being visited), WHITE(not visited) and BLACK(finished visiting).

In [ ]:
private final int GRAY = -1;
private final int WHITE = 0;
private final int BLACK = 1;

public void dfs(int i, byte[] color, IntConsumer c) {
    c.accept(i);
    color[i] = GRAY;
    for(Integer j: outEdges(i)) {
        if(color[j] == WHITE) {
            dfs(j, color, c);
        }
    }
    color[i] = BLACK;
}

For the above example, the sequence is `6->2->1->3->5->4->0`. We need not make use of colors for doing DFS. The below algorithm illustrates this:

In [ ]:
public void dfs(int i, boolean[] visited, IntConsumer c) {
    c.accept(i);
    visited[i] = true;
    for(Integer j: outEdges(i)) {
        if(visited[j] == false) {
            dfs(j, visited, c);
        }
    }
}

We can also accomplish the same without recursion using stacks:

In [ ]:
public void dfs(int i, boolean[] visited, IntConsumer c) {
    Stack<Integer> s = new Stack<>();
    s.add(i);
    Set<Integer> visited = new HashSet<>();
    visited.add(i);

    while (!s.isEmpty()) {
        int current = s.pop();
        consumer.accept(current);

        for (Integer j: outEdges(current)) {
            if (!visited.contains(j)) {
                s.add(j);
                visited.add(j);
            }
        }
    }
} 

Use of three colors may be required in some algorithms like in finding if there is a cycle in a graph or not. As in the case of BFS, we do need to maintain visited map to avoid infinite loop

## Classification of Graphs
On the basis of edges:
- **Directed graph (digraph):** is a graph in which edges have orientations. A directed graph is a pair $G = (V,E)$ where $V$ is set of vertices and $E$ is set of edges which are ordered pairs of vertices $E=\{(v_a,v_b),...,(v_x,v_y)\}$ and $x \ne y$. Multiple edges (two or more edges having same starting and ending vertices) are not allowed under this definition. This definition also avoids loops (edge that starts and ends at the same vertex.
- **Undirected graph**

On the basis of edge weight:
- Weighted graph (attribute assigned to edges, attribute needs to be quantified and comparable)
- Unweighted graph

A weighted graph having all weights equal is equivalent to unweighted graph. Considering the above classification, a graph can be
- Weighted directed (Splitwise)
- Unweighted directed (Twitter)
- Weighted undirected (Metro line)
- Unweighted undirected (Facebook)

**Simple Graph:**
- has no *self loop*, an edge like $(i,i)$
- has no *multi edge*, only one edge corresponding to a vertex pair

**Clique** is a graph where each vertex is connected to every other vertex.

**Disconnected Graph:** if it is possible to pick two vertices such that there is no path between them, it is a disconnected graph.
![dg](images/b3wJjo4.png)

**Cyclic Graph:** a graph $G$ contains a cycle if there is a path in $G$ such that a vertex is reachable from itself. In other words, there is some some path $v_0, v_1,..., v_k, v_0$ in $G$.

### Problems
**Q 1:** Detect if the given directed graph has cycle. A graph contains a cycle if and only if there is a *backedge* in the graph. A backedge is an edge that is from a node to itself or a node to any of its ancestor (grandparent and above).  
**Answer:** If we do a DFS traversal and encounter a node which we had already traversed through, this means that there is a cycle in the graph. In a directed graph, a cycle is present if and only if a node is seen again before all its descendants have been visited. In other words, if a node has a neighbor which is grey, then there is a cycle (and not when the neighbor is black).

In [ ]:
public boolean hasCycle(Map<Integer, List<Integer>> graph) {
    Map<Integer, Character> visited = new HashMap<>();

    // Checks if any of the connected components have a cycle
    for (Integer start : graph.keySet()) {
        // Do not process hasCycle if the node is GRAY or BLACK (visiting, visited)
        if (!visited.containsKey(start) && hasCycle(graph, visited, start)) {
            return true;
        }
    }

    return false;
}

private boolean hasCycle(Map<Integer, List<Integer>> graph, Map<Integer, Character> visited, int i) {
    visited.put(i, 'G');

    for (int n : graph.get(i)) {
        if (visited.getOrDefault(n, 'W') == 'G') {
            return true;
        }

        if (!visited.containsKey(n) && hasCycle(graph, visited, n)) {
            return true;
        }
    }

    visited.put(i, 'B');
    return false;
}

The reason why we require coloring can be expressed by the below example:  
<img src="images/7bGflD8.png" width="600" height="auto">  

<img src="images/Bp41MQt.png" width="600" height="auto">  

What if the graph we have is undirected? The above solution will not work in this case. Consider the scenario when there are only two nodes connected by undirected edge. The above algorithm will give us True in that case also. So what we do is that we keep track of the nodes parent node (the node from which we traversed to the current node).

In [ ]:
public boolean hasCycle(Map<Integer, List<Integer>> graph) {
    Map<Integer, Character> visited = new HashMap<>();

    // Checks if any of the connected components have a cycle
    for (Integer start : graph.keySet()) {
        // Do not process hasCycle if the node is GRAY or WHITE (visiting, visited)
        if (!visited.containsKey(start) && hasCycle(graph, visited, start, null)) {
            return true;
        }
    }

    return false;
}

private boolean hasCycle(Map<Integer, List<Integer>> graph, Map<Integer, Character> visited, int i, int parent) {
    visited.put(i, 'G');

    for (int n : graph.get(i)) {
        if (visited.getOrDefault(n, 'W') == 'G' && n != parent) {
            return true;
        }

        if (!visited.containsKey(n) && hasCycle(graph, visited, n, i)) {
            return true;
        }
    }

    visited.put(i, 'B');
    return false;
}

It is not really required to make use of colors in this case.

In [ ]:
public boolean hasCycle(Map<Integer, List<Integer>> graph) {
    Set<Integer> visited = new HashSet<>();

    // Checks if any of the connected components have a cycle
    for (Integer start : graph.keySet()) {
        // Do not process hasCycle if the node is already visited
        if (!visited.contains(start) && hasCycle(graph, visited, start, null)) {
            return true;
        }
    }

    return false;
}

private boolean hasCycle(Map<Integer, List<Integer>> graph, Set<Integer> visited, int i, int parent) {
    visited.add(i);

    for (int n : graph.get(i)) {
        if (visited.contains(n) && n != parent) {
            return true;
        }

        if (!visited.contains(n) && hasCycle(graph, visited, n, i)) {
            return true;
        }
    }

    return false;
}

BFS can also be used to detect cycle in a graph directed or undirected.

**Q 2:** Given a `MxN` matrix containing 4 types of entries, `s` denotes starting point, `d` denotes destination, `o` denotes not traversable point and `*` denotes traversable point. Find the shortest distance between start and destination. We can only move up, down, left or right. For example, consider the matrix as:
```
o * o s
* o * *
o * * *
d * * *
```
The shortest path in this case is 6. It may also be possible that we are unable to reach destination. In that case return -1.  
**Answer:** We can represent the above matrix path as a graph and then perform BFS starting from start.

In [2]:
public int shortestPathLength(char[][] matrix) {
    // Find the start point
    int[] start = new int[3]; // x, y, length
    for (int i = 0; i < matrix.length; i++) {
        for (int j = 0; j < matrix[i].length; j++) {
            if (matrix[i][j] == 's') {
                start[0] = i;
                start[1] = j;
                break;
            }
        }
    }

    start[2] = 0; // Starting node has path length 0

    Queue<int[]> queue = new ArrayDeque<>();
    queue.offer(start);
    matrix[start[0]][start[1]] = 'o';

    while (!queue.isEmpty()) {
        int[] popped = queue.poll();
        int x = popped[0];
        int y = popped[1];
        int length = popped[2];

        if (matrix[x][y] == 'd') {
            return length;
        }

        if (x + 1 < matrix.length && matrix[x + 1][y] != 'o') {
            if (matrix[x + 1][y] != 'd') matrix[x + 1][y] = 'o';  // Change to 'o' only if it is not destination
            queue.offer(new int[]{x + 1, y, length + 1});         // Need this because we are mutating original matrix
        }
        if (y + 1 < matrix[x].length && matrix[x][y + 1] != 'o') {
            if (matrix[x][y + 1] != 'd') matrix[x][y + 1] = 'o';
            queue.offer(new int[]{x, y + 1, length + 1});
        }
        if (x - 1 >= 0 && matrix[x - 1][y] != 'o') {
            if (matrix[x - 1][y] != 'd') matrix[x - 1][y] = 'o';
            queue.offer(new int[]{x - 1, y, length + 1});
        }
        if (y - 1 >= 0 && matrix[x][y - 1] != 'o') {
            if (matrix[x][y - 1] != 'd') matrix[x][y - 1] = 'o';
            queue.offer(new int[]{x, y - 1, length + 1});
        }
    }

    return -1;
}

char[][] matrix = {{'o', '*', 'o', 's' },
                   {'*', 'o', '*', '*' },
                   {'o', '*', '*', '*' },
                   {'d', '*', '*', '*' }};
System.out.println(shortestPathLength(matrix));

6


[LeetCode 1091](https://leetcode.com/problems/shortest-path-in-binary-matrix/)  
In general, if we have an unweighted graph, then we make use of BFS, to get the shortest path in a Graph. In addition to the path length, if we want to know all the nodes in the shortest path, then we need to maintain a list of predecessors.

In [ ]:
public void printShortestPath(Map<Integer, List<Integer> graph, int start, int end) {
    Queue<Integer> q = new ArrayDeque<>();
    q.offer(start);
    Set<Integer> visited = new HashSet<>();
    visited.add(start);
    Map<Integer, Integer> pred = new HashMap<>();
    pred.put(start, null);

    while(!q.isEmpty()) {
        Integer popped = q.poll();
        if(popped == end) {
            break;
        }

        for(Integer n: graph.get(popped)) {
            if(!visited.contains(n)) {
                q.offer(n);
                visited.add(n);
                pred.put(n, popped);
            }
        }
    }


    List<Integer> path = new ArrayList<>();
    Integer current = end;
    while(current != null) {
        path.add(current);
        current = pred.get(current);
    }
    Collections.reverse(path);

    System.out.println(path);
}

In the previous matrix problem, we can return the shortest path by:

In [4]:
public List<List<Integer>> shortestPath(char[][] matrix) {
    // Find the start point
    int[] start = new int[2];
    for (int i = 0; i < matrix.length; i++) {
        for (int j = 0; j < matrix[i].length; j++) {
            if (matrix[i][j] == 's') {
                start[0] = i;
                start[1] = j;
                break;
            }
        }
    }

    Map<List<Integer>, List<Integer>> predMap = new HashMap<>();
    predMap.put(List.of(start[0], start[1]), null);

    Queue<int[]> queue = new ArrayDeque<>();
    queue.offer(start);
    matrix[start[0]][start[1]] = 'o';

    List<List<Integer>> path = new ArrayList<>();
    while (!queue.isEmpty()) {
        int[] popped = queue.poll();
        int x = popped[0];
        int y = popped[1];
        List<Integer> current = List.of(x, y);

        if (matrix[x][y] == 'd') {
            path.add(current);
            while ((current = predMap.get(current)) != null) {
                path.add(current);
            }

            return path.reversed();
        }

        if (x + 1 < matrix.length && matrix[x + 1][y] != 'o') {
            if (matrix[x + 1][y] != 'd') matrix[x + 1][y] = 'o';
            queue.offer(new int[]{x + 1, y});
            predMap.put(List.of(x + 1, y), current);
        }
        if (y + 1 < matrix[x].length && matrix[x][y + 1] != 'o') {
            if (matrix[x][y + 1] != 'd') matrix[x][y + 1] = 'o';
            queue.offer(new int[]{x, y + 1});
            predMap.put(List.of(x, y + 1), current);
        }
        if (x - 1 >= 0 && matrix[x - 1][y] != 'o') {
            if (matrix[x - 1][y] != 'd') matrix[x - 1][y] = 'o';
            queue.offer(new int[]{x - 1, y});
            predMap.put(List.of(x - 1, y), current);
        }
        if (y - 1 >= 0 && matrix[x][y - 1] != 'o') {
            if (matrix[x][y - 1] != 'd') matrix[x][y - 1] = 'o';
            queue.offer(new int[]{x, y - 1});
            predMap.put(List.of(x, y - 1), current);
        }
    }

    return null;
}

char[][] matrix = {{'o', '*', 'o', 's' },
                   {'*', 'o', '*', '*' },
                   {'o', '*', '*', '*' },
                   {'d', '*', '*', '*' }};
System.out.println(shortestPath(matrix));

[[0, 3], [1, 3], [2, 3], [3, 3], [3, 2], [3, 1], [3, 0]]


**Q 3:** Given a `MxN` array, where each row can either be 0 or 1, return a new array where each cell represents its distance from a 1 cell. For example, if the array is 
```
0 0 0 1
0 0 1 1
0 1 1 0
```
then return
```
3 2 1 0
2 1 0 0
1 0 0 1
```
**Answer:** The easy way, is to do BFS for each 0 node and hence get the shortest distance to a 1. However there is a better way. Instead of starting from a 0, we can start from a 1. Every immediate neighbour to a 1 cell will have a distance 1, and so on.

In [6]:
public void distanceMap(int[][] matrix) {
    List<Integer[]> q = new ArrayList<>();
    Set<List<Integer>> visited = new HashSet<>();

    // Find all the cells having value set to 1 and add it to
    // the queue - multisource BFS. We'll use the queue for
    // maintaining the distance.
    for (int i = 0; i < matrix.length; i++) {
        for (int j = 0; j < matrix[i].length; j++) {
            if (matrix[i][j] == 1) {
                q.add(new Integer[]{i, j, 0});
                visited.add(List.of(i, j));
            }
        }
    }

    // Iterate till the queue is empty
    while (!q.isEmpty()) {
        Integer[] popped = q.remove(0);
        int i = popped[0];
        int j = popped[1];
        int distance = popped[2];

        matrix[i][j] = distance;

        // Left
        if (j - 1 >= 0 && !visited.contains(List.of(i, j - 1))) {
            q.add(new Integer[]{i, j - 1, distance + 1});
            visited.add(List.of(i, j - 1));
        }

        // Right
        if (j + 1 < matrix[0].length && !visited.contains(List.of(i, j + 1))) {
            q.add(new Integer[]{i, j + 1, distance + 1});
            visited.add(List.of(i, j + 1));
        }

        // Up
        if (i - 1 >= 0 && !visited.contains(List.of(i - 1, j))) {
            q.add(new Integer[]{i - 1, j, distance + 1});
            visited.add(List.of(i - 1, j));
        }

        // Down
        if (i + 1 < matrix.length && !visited.contains(List.of(i + 1, j))) {
            q.add(new Integer[]{i + 1, j, distance + 1});
            visited.add(List.of(i + 1, j));
        }
    }
}

int[][] grid = {{0, 0, 0, 1},
                {0, 0, 1, 1},
                {0, 1, 1, 0}};
distanceMap(grid);
System.out.println(Arrays.deepToString(grid));

[[3, 2, 1, 0], [2, 1, 0, 0], [1, 0, 0, 1]]


[LeetCode 542](https://leetcode.com/problems/01-matrix)

**Q 4:** A matrix of size `MxN` contains these three values 0 - no orange, 1 - fresh orange, 2 - rotten orange. A rotten orange will rot all the adjacent oranges. Find the time taken to rot all oranges. If all oranges cannot rot, return -1.  
**Answer:** Do multi source BFS as in the above problem

In [7]:
public int rottenOranges(int[][] matrix) {
    int totalOranges = 0;

    Queue<int[]> q = new ArrayDeque<>();
    for (int i = 0; i < matrix.length; i++) {
        for (int j = 0; j < matrix[i].length; j++) {
            if (matrix[i][j] == 2) {
                q.offer(new int[]{i, j, 0});
                matrix[i][j] = 0;
                totalOranges++;
            } else if (matrix[i][j] == 1) {
                totalOranges++;
            }
        }
    }

    if (totalOranges == 0) return 0;

    int orangeCount = 0;
    while (!q.isEmpty()) {
        int[] popped = q.poll();
        int x = popped[0];
        int y = popped[1];
        int t = popped[2];

        if (++orangeCount == totalOranges) {
            return t;
        }

        if (x + 1 < matrix.length && matrix[x + 1][y] != 0) {
            q.offer(new int[]{x + 1, y, t + 1});
            matrix[x + 1][y] = 0;
        }
        if (y + 1 < matrix[x].length && matrix[x][y + 1] != 0) {
            q.offer(new int[]{x, y + 1, t + 1});
            matrix[x][y + 1] = 0;
        }
        if (x - 1 >= 0 && matrix[x - 1][y] != 0) {
            q.offer(new int[]{x - 1, y, t + 1});
            matrix[x - 1][y] = 0;
        }
        if (y - 1 >= 0 && matrix[x][y - 1] != 0) {
            q.offer(new int[]{x, y - 1, t + 1});
            matrix[x][y - 1] = 0;
        }
    }

    return -1;
}

int[][] oranges = {
        {2, 1, 1},
        {1, 1, 0},
        {0, 1, 1}};
System.out.println(rottenOranges(oranges));
int[][] oranges2 = {{0}};
System.out.println(rottenOranges(oranges2));

4
0


**Q 5:** Given an undirected tree, find the longest path in the tree.  
**Answer:** If we pick any node in the tree and do BFS from this node, we will reach the node farthest away. Now pick the farthest node and do BFS from there. In this two step process, we have found the longest path in the tree.

In [8]:
public int longestPath(Map<Integer, List<Integer>> graph) {
    int[] first = bfs(graph, 0);
    int[] second = bfs(graph, first[0]);

    return second[1];
}

private int[] bfs(Map<Integer, List<Integer>> graph, int start) {
    Queue<int[]> queue = new ArrayDeque<>();
    Set<Integer> visited = new HashSet<>();
    queue.offer(new int[]{start, 0});
    visited.add(start);

    int lastNode = start;
    int distance = 0;
    while (!queue.isEmpty()) {
        int[] popped = queue.poll();
        lastNode = popped[0];
        distance = popped[1];

        for (int child: graph.get(lastNode)) {
            if (!visited.contains(child)) {
                queue.offer(new int[]{child, distance + 1});
                visited.add(child);
            }
        }
    }

    return new int[]{lastNode, distance};
}

Map<Integer, List<Integer>> graph = Map.of(
        0, List.of(1),
        1, List.of(0,2,6),
        2, List.of(1,3,4,9),
        3, List.of(2),
        4, List.of(2,5),
        5, List.of(4),
        6, List.of(1,7,8),
        7, List.of(6),
        8, List.of(6),
        9, List.of(2)
);
System.out.println(longestPath(graph));

5


**Q 6:** Given a graph, clone it. We represent a node of graph as:

In [ ]:
class UndirectedGraphNode {
    public int label;
    public List<UndirectedGraphNode> neighbors;
    
    public UndirectedGraphNode(int label) {
        this.label = label;
        this.neighbors = new ArrayList<>();
    }
}

**Answer:** Do a BFS and create nodes as required

In [ ]:
public UndirectedGraphNode cloneGraph(UndirectedGraphNode node) {
    // Empty graph? Return null
    if(node == null)
        return null;

    // Create new Queue to facilitate BFS
    Queue<UndirectedGraphNode> q = new ArrayDeque<>();
    q.offer(node);
    Set<UndirectedGraphNode> visited = new HashSet<>();
    visited.add(node);

    // Another Queue for clone
    Queue<UndirectedGraphNode> q_ = new ArrayDeque<>();
    UndirectedGraphNode node_ = new UndirectedGraphNode(node.label);
    q_.offer(node_);

    // HashMap to store newly created nodes
    Map<Integer, UndirectedGraphNode> nodeMap = new HashMap<>();
    nodeMap.put(node_.label, node_);

    while(!q.isEmpty()) {
        UndirectedGraphNode popped = q.poll();
        UndirectedGraphNode popped_ = q_.poll();

        for(UndirectedGraphNode n: popped.neighbors) {
            UndirectedGraphNode n_ = nodeMap.get(n.label);
            if(n_ == null) {
                n_ = new UndirectedGraphNode(n.label);
                nodeMap.put(n_.label, n_);
            }
            popped_.neighbors.add(n_);
            
            if(!visited.contains(n)) {
                q.offer(n);
                visited.add(n);

                q_.add(n_);
            }
        }
    }

    return node_;
}

Or we can do a DFS

In [ ]:
public UndirectedGraphNode cloneGraph(UndirectedGraphNode start) {
    if (start == null) return null;

    Set<UndirectedGraphNode> visited = new HashSet<>();
    HashMap<Integer, UndirectedGraphNode> nodes = new HashMap<>();

    UndirectedGraphNode newStart = new UndirectedGraphNode(start.label);
    nodes.put(newStart.label, newStart);

    dfs(start, newStart, visited, nodes);

    return newStart;
}

public void dfs(UndirectedGraphNode oldNode, UndirectedGraphNode newNode, Set<UndirectedGraphNode> visited,
               HashMap<Integer, UndirectedGraphNode> nodes) {
    visited.add(oldNode);
    for (UndirectedGraphNode n : oldNode.neighbors) {
        if (nodes.get(n.label) != null) {
            newNode.neighbors.add(nodes.get(n.label));
        }

        if (!visited.contains(n)) {
            UndirectedGraphNode temp = new UndirectedGraphNode(n.label);
            nodes.put(temp.label, temp);
            newNode.neighbors.add(temp);
            dfs(n, temp, visited, nodes);
        }
    }
}

[LeetCode 133](https://leetcode.com/problems/clone-graph)

**Q 7** Given a weighted undirected graph having $A$ nodes, a source node $C$ and destination node $D$. Find the shortest distance from $C$ to $D$ and if it is impossible to reach node $D$ from $C$ then return -1.  
**Answer:** In this case, we make use of dummy nodes. For example, graph `A-2->B-1->C` will be converted to `A-1->d0-1->B-1->C`. 

In [9]:
private static class Edge {
    public String start;
    public String end;
    public int weight;

    public Edge(String start, String end, int weight) {
        this.start = start;
        this.end = end;
        this.weight = weight;
    }
}

public int shortestPath(List<Edge> edges, String first, String last) {
    // Get the GCD of edge weights
    int weightGCD = edges.getFirst().weight;
    for (Edge edge : edges) {
        weightGCD = gcd(weightGCD, edge.weight);
    }

    int dummyCounter = 0;
    List<Edge> newEdges = new ArrayList<>();
    Iterator<Edge> iterator = edges.iterator();
    while (iterator.hasNext()) {
        Edge edge = iterator.next();

        if (edge.weight != weightGCD) {
            // Remove this edge
            iterator.remove();

            // Replace edge with multiple edges, each having weight = weightGCD
            int dummyEdgeCount = (edge.weight / weightGCD);
            String start = edge.start;
            for (int i = 0; i < dummyEdgeCount; i++) {
                String end;
                if (i != dummyEdgeCount - 1) {
                    end = "d" + dummyCounter;
                    dummyCounter++;
                } else {
                    end = edge.end;
                }

                Edge temp = new Edge(start, end, weightGCD);
                newEdges.add(temp);

                start = end;
            }
        }
    }
    edges.addAll(newEdges);

    // Form adjacency map
    Map<String, List<String>> graph = new HashMap<>();
    for (Edge edge : edges) {
        graph.putIfAbsent(edge.start, new ArrayList<>());
        graph.get(edge.start).add(edge.end);
        graph.putIfAbsent(edge.end, new ArrayList<>());
        graph.get(edge.end).add(edge.start);
    }

    // Perform bfs
    Queue<Object[]> q = new ArrayDeque<>();
    Set<String> visited = new HashSet<>();
    q.offer(new Object[]{first, 0});
    visited.add(first);

    while (!q.isEmpty()) {
        Object[] popped = q.poll();
        String s = (String) popped[0];
        int d = (int) popped[1];

        if (s.equals(last)) {
            return d * weightGCD;
        }

        for (String next : graph.getOrDefault(s, new ArrayList<>())) {
            if (!visited.contains(next)) {
                q.offer(new Object[]{next, d + 1});
                visited.add(next);
            }
        }
    }

    return -1;
}

public int gcd(int a, int b) {
    return b == 0 ? a : gcd(b, a % b);
}

Edge e1 = new Edge("2", "5", 1);
Edge e2 = new Edge("1", "3", 1);
Edge e3 = new Edge("0", "5", 2);
Edge e4 = new Edge("0", "2", 2);
Edge e5 = new Edge("1", "4", 1);
Edge e6 = new Edge("0", "1", 1);

System.out.println(shortestPath(new ArrayList<>(List.of(e1, e2, e3, e4, e5, e6)), "3", "2"));

4


The above algorithm should also work for directed weighted graphs.

**Q 8** Given a 2d matrix, count the number of islands. An island is formed by contiguos `1`s. For example, if the array is:
```   
[1, 1, 0, 0, 0]
[0, 1, 0, 0, 0]
[1, 0, 0, 1, 1]
[0, 0, 0, 0, 0]
[1, 0, 1, 0, 1] 
```
Then it contains 5 islands (look diagonally as well).  
**Answer** We pick a cell and do BFS from it. The number of times BFS is called is the number of islands

In [ ]:
public int islandCount(int[][] matrix) {
    int islandCount = 0;
    for (int i = 0; i < matrix.length; i++) {
        for (int j = 0; j < matrix[0].length; j++) {
            if (matrix[i][j] == 1) {
                traverseIslands(matrix, i, j);
                islandCount++;
            }
        }
    }

    return islandCount;
}

public void traverseIslands(int[][] map, int x, int y) {
    if(map[x][y] == 0) {
        return;
    }
    
    // convert land to water
    map[x][y] = 0;

    // --- Check in all 8 directions ---
    // Top
    if (x - 1 >= 0) {
        traverseIslands(map, x - 1, y);
    }
    // Bottom
    if (x + 1 < map.length) {
        traverseIslands(map, x + 1, y);
    }
    // Right
    if (y + 1 < map[0].length) {
        traverseIslands(map, x, y + 1);
    }
    // Left
    if (y - 1 >= 0) {
        traverseIslands(map, x, y - 1);
    }
    // Top-right
    if (x - 1 >= 0 && y + 1 < map[0].length) {
        traverseIslands(map, x - 1, y + 1);
    }
    // Top-left
    if (x - 1 >= 0 && y - 1 >= 0) {
        traverseIslands(map, x - 1, y - 1);
    }
    // Bottom right
    if (x + 1 < map.length && y + 1 < map[0].length) {
        traverseIslands(map, x + 1, y + 1);
    }
    // Bottom-left
    if (x + 1 < map.length && y - 1 >= 0) {
        traverseIslands(map, x + 1, y - 1);
    }
}

In the above solution we can replace DFS with BFS.  
[LeetCode 200](https://leetcode.com/problems/number-of-islands)

**Q 9** Given a $N\times N$ chess board. How many steps will a knight need to reach destination cell from starting cell on board?  
**Answer:** Do BFS. At any time we can move in 8 directions.

**Q 10** Given a 2-D board $A$ of size $N \times M$ containing 'X' and 'O', capture all regions surrounded by 'X'. A region is captured by flipping all 'O's into 'X's in that surrounded region. For example:
```
[X, X, X, X]             [X, X, X, X]
[X, O, O, X] converts to [X, X, X, X] 
[X, X, O, X]             [X, X, X, X]
[X, O, X, X]             [X, O, X, X]
```
Another example:
```
[X, O, O]             [X, O, O]
[X, O, X] converts to [X, O, X]
[O, O, O]             [O, O, O]
```
**Answer** We pick all the Os in the boundary and start multi source BFS from the selected sources

In [10]:
public void capture(char[][] matrix) {
    Queue<int[]> q = new ArrayDeque<>();
    
    // Check the boundary for the presence of 'O'
    for (int i = 0; i < matrix.length; i++) {
        // Left boundary
        if (matrix[i][0] == 'O') {
            matrix[i][0] = '-';
            q.offer(new int[]{i, 0});
        }
        // Right boundary
        if (matrix[i][matrix[0].length - 1] == 'O') {
            matrix[i][matrix[0].length - 1] = '-';
            q.offer(new int[]{i, matrix[0].length - 1});
        }
    }
    for (int j = 0; j < matrix[0].length; j++) {
        // Top boundary
        if (matrix[0][j] == 'O') {
            matrix[0][j] = '-';
            q.offer(new int[]{0, j});
        }
        // Bottom boundary
        if (matrix[matrix.length - 1][j] == 'O') {
            matrix[matrix.length - 1][j] = '-';
            q.offer(new int[]{matrix.length - 1, j});
        }
    }
    
    while (!q.isEmpty()) {
        int[] popped = q.poll();
        int x = popped[0], y = popped[1];

        // Top
        if (x - 1 >= 0 && matrix[x - 1][y] == 'O') {
            matrix[x - 1][y] = '-';
            q.offer(new int[]{x - 1, y});
        }
        // Bottom
        if (x + 1 < matrix.length && matrix[x + 1][y] == 'O') {
            matrix[x + 1][y] = '-';
            q.offer(new int[]{x + 1, y});
        }
        // Right
        if (y + 1 < matrix[0].length && matrix[x][y + 1] == 'O') {
            matrix[x][y + 1] = '-';
            q.offer(new int[]{x, y + 1});
        }
        // Top
        if (y - 1 >= 0 && matrix[x][y - 1] == 'O') {
            matrix[x][y - 1] = '-';
            q.offer(new int[]{x, y - 1});
        }
    }
    
    // Traverse over the matrix and transform all remaining 'O' to 'X'
    // These 'O' were covered fully by 'X'
    for (int i = 0; i < matrix.length; i++) {
        for (int j = 0; j < matrix[0].length; j++) {
            if (matrix[i][j] == 'O') {
                matrix[i][j] = 'X';
            } else if (matrix[i][j] == '-') {
                matrix[i][j] = 'O';
            }
        }
    }
}

char[][] board = {{'X','X','X','X'},
                  {'X','O','O','X'},
                  {'X','X','O','X'},
                  {'X','O','X','X'}};
capture(board);
System.out.println(Arrays.deepToString(board));

[[X, X, X, X], [X, X, X, X], [X, X, X, X], [X, O, X, X]]


[LeetCode 130](https://leetcode.com/problems/surrounded-regions)

## Topological Sort
Relevant only to Directed Acyclic Graph, is linear ordering of vertices in graph such that if there is an edge $(u, v)$ then $u$ appears before $v$ in the order. There can be multiple correct topological sort order for the same graph.

In [ ]:
public int[] topologicalSort(List<List<Integer>> graph) {
    int[] ordered = new int[graph.size()];
    int index = ordered.length - 1;

    Set<Integer> visited = new HashSet<>();
    for (int i = 0; i < graph.size(); i++) {
        // Perform dfs from every unvisited node
        if (!visited.contains(i)) {
            index = topologicalSort(graph, index, i, ordered, visited);
        }
    }

    return ordered;
}

private int topologicalSort(List<List<Integer>> graph, int index, int start, int[] ordered, Set<Integer> visited) {
    visited.add(start);

    for (Integer n : graph.get(start)) {
        if (!visited.contains(n)) {
            index = topologicalSort(graph, index, n, ordered, visited);
        }
    }

    ordered[index] = start;
    return index - 1;
}

Topological sort is used for multiple purposes:
- code compilation order based on source file dependencies
- task scheduling where some tasks must be completed before others
- linearizing commit history in Git

## Dijkstra Algorithm
To find the shortest path in a weighted graph, we use Dijkstra Algorithm which gives the shortest path from a source node to all the other nodes from the source node.

In [18]:
public List<Integer> shortestPath(Map<Integer, List<Integer[]>> graph, int start, int end) {
    // Priority queue: [node, distance]
    PriorityQueue<int[]> pq = new PriorityQueue<>((a, b) -> a[1] - b[1]);

    // Track shortest distance to each node
    Map<Integer, Integer> distances = new HashMap<>();

    // Track parent of each node for path reconstruction
    Map<Integer, Integer> parent = new HashMap<>();

    // Track visited nodes
    Set<Integer> visited = new HashSet<>();

    // Initialize
    pq.offer(new int[]{start, 0});
    distances.put(start, 0);
    parent.put(start, null);

    while (!pq.isEmpty()) {
        int[] current = pq.poll();
        int node = current[0];
        int dist = current[1];

        // Skip if already visited
        if (visited.contains(node)) {
            continue;
        }

        visited.add(node);

        // If we reached the end, reconstruct path
        if (node == end) {
            return reconstructPath(parent, start, end);
        }

        // Explore neighbors
        for (Integer[] neighbor : graph.getOrDefault(node, new ArrayList<>())) {
            int nextNode = neighbor[0];
            int weight = neighbor[1];

            if (visited.contains(nextNode)) {
                continue;
            }

            int newDist = dist + weight;

            // If found shorter path, update
            if (newDist < distances.getOrDefault(nextNode, Integer.MAX_VALUE)) {
                distances.put(nextNode, newDist);
                parent.put(nextNode, node);
                pq.offer(new int[]{nextNode, newDist});
            }
        }
    }

    // No path found
    return new ArrayList<>();
}

private List<Integer> reconstructPath(Map<Integer, Integer> parent, int start, int end) {
    List<Integer> path = new ArrayList<>();
    Integer current = end;

    while (current != null) {
        path.add(current);
        current = parent.get(current);
    }

    Collections.reverse(path);

    // Verify path is valid
    if (path.get(0) != start) {
        return new ArrayList<>(); // No valid path
    }

    return path;
}

// Graph representation: Map<Node, List<[neighbor, weight]>>
Map<Integer, List<Integer[]>> graph = new HashMap<>();

// Add edges
graph.put(0, List.of(new Integer[]{1, 4}, new Integer[]{2, 1}));
List<Integer[]> e = new ArrayList<>(); e.add(new Integer[]{3, 1});
graph.put(1, e);
graph.put(2, List.of(new Integer[]{1, 2}, new Integer[]{3, 5}));
graph.put(3, List.of());

List<Integer> path = shortestPath(graph, 0, 3);
System.out.println(path);

[0, 2, 1, 3]


## Disjoint Set
Suppose we have $N$ nodes, all unconnected. This means that we have $N$ connected components. We can form a undirected edge between any two nodes. We call this $union(x, y)$ operation. It can be illustrated by diagrams below:

Initially all there is nt edge between any two vertices:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/039a333.jpg" width="600" height="auto">

We use array `Arr` to represent connectivity. Indices of this array represent the nodes and `Arr[A] == Arr[B]` then `A` and `B` nodes are connected
<img src="https://he-s3.s3.amazonaws.com/media/uploads/1539ad6.jpg" width="600" height="auto">

On performing $union(2, 1)$, we get:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/32f6a91.jpg" width="600" height="auto">  

On performing $union(4, 3)$, $union(8,4)$ and $union(9,3)$ we get:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/4c11a99.jpg" width="600" height="auto">
<img src="https://he-s3.s3.amazonaws.com/media/uploads/6a7bc9a.jpg" width="600" height="auto"><br>  

After performing $union(6,5)$, we get:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/66d9b5d.jpg" width="600" height="auto">
<img src="https://he-s3.s3.amazonaws.com/media/uploads/7439d01.jpg" width="600" height="auto"><br>  

We can define another operation $find(x,y)$ which will return true if nodes $x$ and $y$ are in the same connected component. In the above graph, $find(0,7)$ will be false. Now if we do $union(5,2)$, we get:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/a7f5551.jpg" width="600" height="auto">
<img src="https://he-s3.s3.amazonaws.com/media/uploads/8538800.jpg" width="600" height="auto"><br>  

In total, we are left with 4 sets or 4 connected components.

In [5]:
def init(arr, N):
    for i in range(N):
        arr.append(i)
        
def find(arr, x, y):
    return arr[x] == arr[y]

def union(arr, N, x, y):
    if not find(arr, x, y):
        temp = arr[x]
        for i in range(N):
            if arr[i] == temp:
                arr[i] = arr[y]
            
N = 10
arr = []
init(arr, N)

union(arr,N,2,1); union(arr,N,4,3); union(arr,N,8,4); union(arr,N,9,3); union(arr,N,6,5); union(arr,N,5,2)
print(arr)

[0, 1, 1, 3, 3, 1, 1, 7, 3, 3]


There is a better approach for the $union$ operation. Here the term $Arr[A]$ contains the parent node of A. Consider 6 nodes. Initially, since all nodes are unconnected, we have:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/fd8e878.jpg" width="600" height="auto">  

Also we define a new operation $root(x)$ which will give the root node of a node $x$. Once we do $union(1,0)$, the node 0 becomes root node of node 1.  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/1ddeb33.jpg" width="600" height="auto">  

$union(0,2)$:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/201a115.jpg" width="600" height="auto"> 

$union(3,4)$:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/4e3e47b.jpg" width="600" height="auto"> 

$union(1,4)$:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/62016dc.jpg" width="600" height="auto"> 

In [ ]:
def root(arr, x):
    while arr[x] != x:
        x = arr[x]
        
    return x

If the tree formed in each case is balanced, then it would be much better. The find operation will take $O(logN)$ time complexity. So whenever, we connect two root nodes, we connect the smaller component to the larger connected component.  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/a1f5858.jpg" width="600" height="auto">

Illustrated example:  
<img src="https://he-s3.s3.amazonaws.com/media/uploads/d95f145.jpg" width="600" height="auto">
<img src="https://he-s3.s3.amazonaws.com/media/uploads/e8364d0.jpg" width="600" height="auto">
<img src="https://he-s3.s3.amazonaws.com/media/uploads/ff4f029.jpg" width="600" height="auto">

In [ ]:
# We make use of a size array which will contain size of tree rooted at i
def init(arr, size, N):
    for i in range(N):
        arr.append(i)
        size.append(1)
        
def root(arr, x):
    while arr[x] != x:
        x = arr[x]
        
    return x

def find(arr, x, y):
    return root(arr, x) == root(arr, y)
    
def union(arr, size, N, x, y):
    root_x = root(arr, x)
    root_y = root(arr, y)
    
    if size[root_x] >= size[root_y]:
        arr[root_y] = root_x
        size[root_x] += size[root_y]
    else:
        arr[root_x] = root_y
        size[root_y] += size[root_x]

What if the nodes are not numeric? Just use a map instead of array

In [13]:
arr = {}
size = {}
vertices = ['a', 'b', 'c', 'd']

def init(arr, size, vertices):
    for v in vertices:
        arr[v] = v
        size[v] = 1
        
def root(arr, x):
    while arr[x] != x:
        x = arr[x]
        
    return x

def find(arr, x, y):
    return root(arr, x) == root(arr, y)

def union(arr, size, x, y):
    root_x = root(arr, x)
    root_y = root(arr, y)
    
    if size[root_x] >= size[root_y]:
        arr[root_y] = root_x
        size[root_x] += size[root_y]
    else:
        arr[root_x] = root_y
        size[root_y] += size[root_x]
        
init(arr, size, vertices)
union(arr, size, 'a', 'b'); union(arr, size, 'c', 'd')
print(find(arr, 'a', 'c'))
print(find(arr, 'a', 'b'))

False
True


### Kruskal's Algorithm
Given a connected and undirected graph, a **spanning tree** of that graph is a subgraph that is a tree and connects all the vertices together. A **minimum spanning tree (MST)** of a connected and undirected graph is a spanning tree with weight less than or equal to the weight of every other spanning tree. 

Kruskal's algorithm finds the MST. Given a set of vertices and edges of a weighted undirected tree, we sort the edges in ascending order. Each time we pick an edge such that picking that edge doesn't form a cycle.

In [14]:
# Vertex count and edges
vertices = 4
edges = [(0, 1, 10), (0, 2, 6) ,(0, 3, 5), (1, 3, 15) ,(2, 3, 4)]

# Finds root of a vertex x
def root(arr, x):
    while arr[x] != x:
        x = arr[x]
        
    return x

# Do union operation
def union(arr, size, N, x, y):
    root_x = root(arr, x)
    root_y = root(arr, y)
    
    if size[root_x] >= size[root_y]:
        arr[root_y] = root_x
        size[root_x] += size[root_y]
    else:
        arr[root_x] = root_y
        size[root_y] += size[root_x]
        
def kruskal(N, edges):
    result = []
    
    # Sort the edges
    g = edges
    g = sorted(g, key=lambda x: x[2])
    
    arr = []
    size = []
    
    # Form arr and size arrays
    for i in range(N):
        arr.append(i)
        size.append(1)
        
    # Number of edges in MST will be N-1
    j = 0
    k = 0
    while j < N-1:
        f, t, w = g[k]
        
        root_f = root(arr, f)
        root_t = root(arr, t)
        
        if root_f != root_t:
            result.append((f, t, w))
            j += 1
            union(arr, size, N, f, t)
            
        k += 1
        
    min_cost = 0
    for e in result:
        min_cost += e[2]
        
    return min_cost

print(kruskal(4, edges))

19


Time Complexity is $O(ElogE)$ or $O(ElogV)$. Sorting of edges takes $O(ELogE)$ time. After sorting, we iterate through all edges and apply find-union algorithm. The find and union operations can take atmost $O(LogV)$ time. So overall complexity is $O(ELogE + ELogV)$ time. The value of $E$ can be atmost $O(V^2)$, so $O(LogV)$ are $O(LogE)$ same. Therefore, overall time complexity is $O(ElogE)$ or $O(ElogV)$